## Project Requirements:
Scrape profiles and get:
1. the number of posts
2. the follower count
3. number of likes on most liked post
4. numbner of likes

## Follow Up
Save each profile in its own data frame with each row corresponding to a post <br>
export as a excel file with each profile as a worksheet <br>
find a way to edit so that the sheet includes some global account variables (ex. number of followers) outside the dataframe

In [2]:
from instascrape import Profile, scrape_posts, Post
from selenium.webdriver import Chrome
from datetime import datetime
import pandas as pd
import numpy as np
import pickle
from openpyxl.workbook.child import INVALID_TITLE_REGEX
from openpyxl import workbook
import re
from utils import check_valid_username, write_to_excel, get_top_post, create_post_df
from pathlib import Path
from tqdm import tqdm

In [3]:
cwd = Path.cwd()

#mac
#chrome_driver_path = cwd / 'chromedriver'

#windows
chrome_driver_path = cwd / 'chromedriver.exe'
driver = Chrome(chrome_driver_path)


In [4]:
headers = {
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36 Edg/87.0.664.57",
    "cookie": "sessionid=7320119797%3AbistHHaSBGycwv%3A2"
}

In [5]:
users = pd.read_csv("Social_Media_Study_Handles.csv")
users = users.reindex(columns=[*users.columns, 'follower_count', 'number_of_posts'])

#Specifying datatype for each columns
users['School'] = users['School'].astype(object)
users['IG Username'] = users['IG Username'].astype(object)
users['follower_count'] = users['follower_count'].astype('Int64')
users['number_of_posts'] = users['number_of_posts'].astype('Int64')

# users = users.reindex(columns=[*users.columns, 'follower_count', 'number_of_posts', 'top_post_likes', 
# 'top_post_date_posted', 'top_post_url', 'first_post_likes', 'first_post_date_posted', 'first_post_url'])
# This code is no longer userful since we are storing each account's posts in its own dataframe
# users['top_post_likes'] = users['top_post_likes'].astype('Int64')
# users['top_post_date_posted'] = users['top_post_date_posted'].astype('datetime64[ns]')
# users['top_post_url'] = users['top_post_url'].astype(object)
# users['first_post_likes'] = users['first_post_likes'].astype('Int64')
# users['first_post_date_posted'] = users['first_post_date_posted'].astype('datetime64[ns]')
# users['first_post_url'] = users['first_post_url'].astype(object)

In [6]:
#test purposes
# users = users.head(5).append(users.iloc[89])

#List of dataframes representing the posts of each user
users_df = []

#indicies of all users with more than 200 posts
index_greater_200 = []

#indicies of all users with less than 200 posts
index_200_or_less = []

for i in tqdm(range(0, len(users))):
    if (not pd.isnull(users.iloc[i]['IG Username'])):
        #Scrape the profile
        print(users.iloc[i]['IG Username'])
        profile = Profile(users.iloc[i]['IG Username'])
        profile.scrape(headers=headers)

        #Adding profile datapoints to dataframe
        users.at[i, 'follower_count'] = profile.followers
        users.at[i, 'number_of_posts'] = profile.posts

        #Scraping Posts

        posts = profile.get_posts(webdriver=driver)
        print(len(posts))

        #account has more than 200 posts
        if (len(posts) > 200):
            print("{} with username {} has {} posts, more than 200".format(users.iloc[i]['School'], users.iloc[i]['IG Username'], len(posts)))
            index_greater_200.append(i)
        
        else:
            index_200_or_less.append(i)
            try:
                scraped_posts, unscraped = scrape_posts(posts, webdriver=driver, silent=False, headers=headers, pause=10)
                
                #Appending dataframe from account to each 
                users_df.append(create_post_df(scraped_posts))

            except Exception as inst:
                print("an error has occured on {}".format(users.iloc[i]['IG Username']))
                print(inst)
                users_df.append(create_post_df([]))
        
    
    # if no username, we're just going to add an empty dataframe
    else:
        users_df.append(create_post_df([]))

    #Writing test dataframes to pickle object for easier debugging in the future
    if (i % 10 == 0 and i != 0):
        with open('test_dataframes_{}-{}.obj'.format(i-10, i), 'wb') as f:
            pickle.dump((users_df, users), f)
        
        
        
        #old code from just getting top and first posts
        # top_post = get_top_post(scraped_posts)

        # #Post are scraped from most recent to oldest, therefore, the earliest post will be the last one scraped in theory
        # first_post = scraped_posts[-1]

        # users.at[i, 'top_post_likes'] = top_post.likes
        # users.at[i, 'top_post_date_posted'] = datetime.fromtimestamp(top_post.timestamp)
        # users.at[i, 'top_post_url'] = "instagram.com/p/{}".format(top_post.shortcode)
        # users.at[i, 'first_post_likes'] = first_post.likes
        # users.at[i, 'first_post_date_posted'] = datetime.fromtimestamp(first_post.timestamp)
        # users.at[i, 'first_post_url'] = "instagram.com/p/{}".format(first_post.shortcode)


  0%|          | 0/121 [00:00<?, ?it/s]

willseyeresidents
33
0: CDABF2hhEhP - 2020-07-23 17:49:11
1: CC6_LwDhnTu - 2020-07-21 18:56:19
2: CC4Kk8ZBGIc - 2020-07-20 16:38:09
3: CCtc94uh_SW - 2020-07-16 12:47:12
4: CCqrp49B5j1 - 2020-07-15 10:57:49
5: CCjOYsGBwWO - 2020-07-12 13:27:23
6: CCecas4hdY- - 2020-07-10 16:53:47
7: CCHSdAehmf7 - 2020-07-01 17:04:11
8: CB3xPaehw43 - 2020-06-25 16:25:22
9: CBy5lzbBlq8 - 2020-06-23 19:02:08
10: CBwQp8qhq3y - 2020-06-22 18:25:57
11: CBtTXidBM4k - 2020-06-21 14:51:56
12: CBqW91ohOfD - 2020-06-20 11:25:39
13: CBoCRLXhQex - 2020-06-19 13:46:18
14: CBlvolqAn4t - 2020-06-18 16:25:00
15: CBjLzmHBJxE - 2020-06-17 16:33:27
16: CANgGIPAQc0 - 2020-05-15 09:56:03
17: CALFcpLAZq1 - 2020-05-14 11:24:43
18: B_lFRQnBz3j - 2020-04-29 17:12:01
19: B-sAg1yhXLQ - 2020-04-07 13:13:45
20: B9Z5wFmBtvD - 2020-03-06 14:56:56
21: B9HGomcBRkn - 2020-02-28 07:43:57
22: B8nbgg2hDsc - 2020-02-16 00:30:39
23: B7yyPaVB9wF - 2020-01-26 13:49:32
24: B6UcX8CB3J_ - 2019-12-20 22:29:51
25: B6TRZBrBHoW - 2019-12-20 11:34:38
2

  1%|          | 1/121 [05:56<11:52:12, 356.10s/it]

32: B4BZYIpAuYT - 2019-10-24 20:55:18
bascompalmereye


  2%|▏         | 2/121 [06:30<5:30:59, 166.89s/it] 

237
Bascom Palmer/University of Miami with username bascompalmereye has 237 posts, more than 200
wilmereyeresidents
34
0: CVYQ03DrANh - 2021-10-23 13:13:33
1: CU-eULLAyeZ - 2021-10-13 12:51:10
2: CUKkD_-AGmc - 2021-09-23 09:00:53
3: CTLCjN9rd5C - 2021-08-29 16:55:53
4: CRwRrpghB4P - 2021-07-25 10:57:13
5: CRIGjk4B2bf - 2021-07-09 20:30:23
6: CRDNMZyhTKq - 2021-07-07 22:52:11
7: CQjuxxIB98Y - 2021-06-25 17:29:57
8: CMtBiTAhjET - 2021-03-21 22:01:27
9: CKzaswbhAF7 - 2021-02-02 15:33:59
10: CKJ7EzWh1yB - 2021-01-17 12:48:47
11: CJjWki4hcnY - 2021-01-02 13:18:40
12: CJKFwOuBfrA - 2020-12-23 17:50:42
13: CI7S8ulBBXg - 2020-12-17 23:57:24
14: CHHOF0HBjtU - 2020-11-02 22:03:02
15: CEK--gvn7tR - 2020-08-21 20:33:45
16: CEE7Gownxu_ - 2020-08-19 12:04:28
17: CD0FBcwh8Jk - 2020-08-12 23:04:03
18: CDNY5mJhJgz - 2020-07-28 22:27:21
19: CCsEnnzBMPi - 2020-07-15 23:55:12
20: CCfNRLEhLAi - 2020-07-11 00:00:39
21: CCFTbxKBA5w - 2020-06-30 22:34:17
22: CCAPSKrhGBC - 2020-06-28 23:21:49
23: CB15aM6Hy3F -

  2%|▏         | 3/121 [12:34<8:25:08, 256.85s/it]

33: CAJHZJ-AtfL - 2020-05-13 17:03:14
uiowaeye
131
0: CVGjF6MLdlJ - 2021-10-16 16:06:50
1: CU5gfRpLtyq - 2021-10-11 14:33:57
2: CUz3DYRgbI1 - 2021-10-09 09:55:41
3: CSFgrMYrfZY - 2021-08-02 16:52:17
4: CR4eaSih6Rh - 2021-07-28 15:22:22
5: CRZ9tDLBZO4 - 2021-07-16 18:59:22
6: CNDcat-hs1J - 2021-03-30 14:59:38
7: CMVMAzah-wd - 2021-03-12 14:51:13
8: CLxXNG7h8TX - 2021-02-26 16:57:51
9: CLxWUtIhCmI - 2021-02-26 16:48:39
10: CLCioVhhEos - 2021-02-08 12:31:53
11: CKwbZApBXJp - 2021-02-01 11:42:18
12: CKSH8vMhObS - 2021-01-20 17:15:12
13: CJeV2zyBa35 - 2020-12-31 14:36:13
14: CJEFs4ahll6 - 2020-12-21 09:54:48
15: CIZZd9ZhfZC - 2020-12-04 20:00:11
16: CIHJ_1tBmqf - 2020-11-27 17:58:40
17: CH1Wo83BDSc - 2020-11-20 20:02:48
18: CHjUSibhiWR - 2020-11-13 19:55:56
19: CHQrqnvBzA5 - 2020-11-06 14:14:38
20: CHCF1n1hDTh - 2020-10-31 23:14:43
21: CG0psvnhHze - 2020-10-26 17:58:43
22: CGs_jM6BeuV - 2020-10-23 18:35:44
23: CGbEc9Ohm1b - 2020-10-16 19:32:14
24: CGIXPQEBYIL - 2020-10-09 13:10:49
25: CF2ap

In [ ]:
write_to_excel(users_df, users.iloc[index_200_or_less])

TypeError: len() takes exactly one argument (2 given)

In [13]:
#Writing test dataframes to pickle object for easier debugging in the future
with open('test_dataframes.obj', 'wb') as f:
    pickle.dump((users_df, users), f)

In [3]:
with open('test_dataframes.obj', 'rb') as f:
    loaded = pickle.load(f)

users_df = loaded[0]
users = loaded[1]
users = users.reset_index()